In [1]:
import psycopg2
import arcgis
from arcgis.gis import GIS
from arcgis.features import FeatureLayer, FeatureLayerCollection, GeoAccessor
import sys
sys.path.append('../')
sys.path.append('../../')
from utils import get_config
from gis_resources import get_lat_long
import pandas as pd

### Objective
* Read farmersmarket_2023-2593956 table from postgres.  
* Using the lat / long in the table, create a point layer.  

In [2]:
gis = GIS("https://ucsdonline.maps.arcgis.com/home", client_id=get_config("arcgis","clientid"))

Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://ucsdonline.maps.arcgis.com/sharing/rest/oauth2/authorize?response_type=code&client_id=Elm5V3upnnV17Q3r&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=GfKJHiOQgZbGyRKnqqMbCldNAvdGFE&allow_verification=false
Enter code obtained on signing in using SAML: ········


/Users/amolkale/Documents/DSE/DSE260-Capstone-Project/capstone/venv3.9/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ucsdonline.maps.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [3]:
nourish_user = get_config("nourish_db","username")
nourish_pswd = get_config("nourish_db","passkey")
conn = psycopg2.connect(
    host="awesome-hw.sdsc.edu",
    database="nourish",
    user=nourish_user,
    password=nourish_pswd)

In [4]:
%%time
# create a cursor
cur = conn.cursor()
        

# execute a statement
cur.execute('select listing_name, location_Address, longitude, latitude from "farmersmarket_2023-2593956"')

        # display the PostgreSQL database server version
results = cur.fetchall()
       
	# close the communication with the PostgreSQL
cur.close()

CPU times: user 4.6 ms, sys: 5.07 ms, total: 9.66 ms
Wall time: 121 ms


In [5]:
results[1]

('Market in The Park',
 '300 Conti, Mobile, Alabama 36602',
 Decimal('-88.045015'),
 Decimal('30.69035'))

In [6]:
farmers_market_list = [(row[0],row[1],row[2],row[3]) for row in results]
farmers_market_df = pd.DataFrame(farmers_market_list,
              columns=("listing_name","location_address", "longitude","latitude")
              );
print(f"Farmers Market SHape: {farmers_market_df.shape}")
farmers_market_df.head(3)

Farmers Market SHape: (1058, 4)


,listing_name,location_address,longitude,latitude
0,Colorado Farm and Art Market,"7350 Pine Creek Road, Colorado Springs, Colora...",-104.81468,38.937716
1,Market in The Park,"300 Conti, Mobile, Alabama 36602",-88.045015,30.69035
2,The Territory Farmers' Market,"2000 Territory Drive, Galena, Illinois 61036",-90.31353,42.404443


In [7]:
farmers_market_df[farmers_market_df.isna()["longitude"]]

,listing_name,location_address,longitude,latitude
921,Historic Lewes Farmers Market,None,None,None
1042,Greater Canonsburg Chamber of Commerce Farmer’...,None,None,None


In [8]:
farmers_market_df = farmers_market_df[farmers_market_df['longitude'].notna()]
farmers_market_df = farmers_market_df[farmers_market_df['latitude'].notna()]

In [10]:
# gdf = geopandas.GeoDataFrame(farmers_market_df, 
#                              geometry=geopandas.points_from_xy(farmers_market_df.longitude, farmers_market_df.latitude))


In [33]:
# farmers_market_sedf = GeoAccessor.from_geodataframe(gdf, inplace=False, column_name='SHAPE')
# print(f"Farmers Market SEDF Shape: {farmers_market_sedf.shape}")
# farmers_market_sedf.head(3)

Somehow I am unable to convert the latitude and longitudes to a spatially enabled dataframe,so I am going to geocode the address available.

In [38]:
# farmers_market_sedf = GeoAccessor.from_df(df=farmers_market_df,address_column="location_address")
# print(f"Farmers Market SEDF Shape: {farmers_market_sedf.shape}")
# farmers_market_sedf.head(3)

In [11]:
farmers_market_df[['longitude','latitude']] = farmers_market_df.apply(lambda row: get_lat_long(row.location_address), axis='columns', result_type='expand')
# print("Shape:",farmers_market_df.shape)
# farmers_market_df.head(5)



In [13]:
farmers_market_df

,listing_name,location_address,longitude,latitude
0,Colorado Farm and Art Market,"7350 Pine Creek Road, Colorado Springs, Colora...",-104.813284,38.937974
1,Market in The Park,"300 Conti, Mobile, Alabama 36602",-88.044585,30.690215
2,The Territory Farmers' Market,"2000 Territory Drive, Galena, Illinois 61036",-90.317574,42.399287
3,City of Plymouth Farmers Market,"15500 County Road 6, Plymouth, Minnesota 55447",-93.476260,44.996380
4,Durham Farmers' Market,"501 Foster Street, Durham, North Carolina 27701",-78.901498,36.000645
...,...,...,...,...
1053,Bexley Farmers Market,"South Dawson Avenue & East Main Street, Bexley...",-82.935720,39.957140
1054,2nd Street Market,"600 East 2nd Street, Dayton, OH, USA",-84.180817,39.762778
1055,Farmers Market Centerville Texas,"114 E St Marys St, Centerville, TX, USA",-95.978552,31.258045
1056,Downtown Beloit Farmers' Market - Winter,"557 E Grand Ave, Beloit, WI, USA",-89.031517,42.499645


In [14]:
farmers_market_sedf = GeoAccessor.from_xy(farmers_market_df.copy(),x_column="longitude",y_column="latitude")
print(f"Farmers Market SEDF Shape: {farmers_market_sedf.shape}")
farmers_market_sedf.head(3)

Farmers Market SEDF Shape: (1056, 5)


,listing_name,location_address,longitude,latitude,SHAPE
0,Colorado Farm and Art Market,"7350 Pine Creek Road, Colorado Springs, Colora...",-104.813284,38.937974,"{""spatialReference"": {""wkid"": 4326}, ""x"": -104..."
1,Market in The Park,"300 Conti, Mobile, Alabama 36602",-88.044585,30.690215,"{""spatialReference"": {""wkid"": 4326}, ""x"": -88...."
2,The Territory Farmers' Market,"2000 Territory Drive, Galena, Illinois 61036",-90.317574,42.399287,"{""spatialReference"": {""wkid"": 4326}, ""x"": -90...."


In [20]:
farmers_market_sedf.rename(columns={'listing_name':'name','location_address':'address'}, inplace=True)

In [24]:
farmers_market_sedf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1056 entries, 0 to 1057
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   name       1056 non-null   object  
 1   address    1056 non-null   object  
 2   longitude  1056 non-null   float64 
 3   latitude   1056 non-null   float64 
 4   SHAPE      1056 non-null   geometry
dtypes: float64(2), geometry(1), object(2)
memory usage: 49.5+ KB


In [21]:
%%time
# Convert back from a SEDF into a feature layer, and publishing on AGOL
feature_layer = farmers_market_sedf.spatial.to_featurelayer(title="Farmers Market 2023", 
                                                         gis=gis, 
                                                         folder='nourish_gis',
                                                         tags=['FarmersMarket'],
                                                         service_name='farmers_market_2023_pt_lyr')

CPU times: user 925 ms, sys: 134 ms, total: 1.06 s
Wall time: 11.7 s


In [22]:
feature_layer

<Item title:"Farmers Market 2023" type:Feature Layer Collection owner:akale_UCSDOnline>